In [1]:
import tensorflow as tf
from transformers import BertTokenizer, TFBertForMaskedLM, DataCollatorForLanguageModeling
from datasets import load_dataset
from transformers import TFTrainer, TFTrainingArguments

C:\ProgramData\miniconda3\envs\tensorflow-gpu\lib\site-packages\numpy\_distributor_init.py:30: UserWarning: loaded more than 1 DLL from .libs:
C:\ProgramData\miniconda3\envs\tensorflow-gpu\lib\site-packages\numpy\.libs\libopenblas.GK7GX5KEQ4F6UYO3P26ULGBQYHGQO7J4.gfortran-win_amd64.dll
C:\ProgramData\miniconda3\envs\tensorflow-gpu\lib\site-packages\numpy\.libs\libopenblas64__v0.3.21-gcc_10_3_0.dll
  warnings.warn("loaded more than 1 DLL from .libs:"


In [2]:
tf_device = '/GPU:0' if tf.config.list_physical_devices('GPU') else '/CPU:0'
print(tf_device)

/GPU:0


In [3]:
# Load the tokenizer
tokenizer = BertTokenizer.from_pretrained("bert-base-german-cased")

In [4]:
def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True, max_length=512)

def load_and_prepare_dataset(dataset_path):
    dataset = load_dataset('text', data_files={'train': dataset_path})

    tokenized_datasets = dataset.map(tokenize_function, batched=True)
    data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer, mlm=True, mlm_probability=0.15
    )
    tf_dataset = tokenized_datasets["train"].to_tf_dataset(
    columns=["input_ids", "attention_mask", "token_type_ids"],
    shuffle=True,
    batch_size=16,
    collate_fn=data_collator
    )
    return tf_dataset

In [5]:
dataset_path = '../dataset/clean_text_data.txt'
tf_dataset = load_and_prepare_dataset(dataset_path)

Map:   0%|          | 0/76697 [00:00<?, ? examples/s]

In [6]:
with tf.device(tf_device):
    model = TFBertForMaskedLM.from_pretrained("bert-base-german-cased")

All PyTorch model weights were used when initializing TFBertForMaskedLM.

All the weights of TFBertForMaskedLM were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertForMaskedLM for predictions without further training.


In [7]:
training_args = TFTrainingArguments(
    output_dir="./bert-german-checkpoints",
    overwrite_output_dir=True,
    num_train_epochs=3,
    per_device_train_batch_size=16,
    save_steps=10_000,
    save_total_limit=2
)

In [8]:
with tf.device(tf_device):
    trainer = TFTrainer(
        model=model,
        args=training_args,
        train_dataset=tf_dataset
    )

C:\ProgramData\miniconda3\envs\tensorflow-gpu\lib\site-packages\transformers\trainer_tf.py:118: FutureWarning: The class `TFTrainer` is deprecated and will be removed in version 5 of Transformers. We recommend using native Keras instead, by calling methods like `fit()` and `predict()` directly on the model object. Detailed examples of the Keras style can be found in our examples at https://github.com/huggingface/transformers/tree/main/examples/tensorflow
  warnings.warn(


In [9]:
trainer.train()

INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:GPU:0', '/job:localhost/replica:0/task:0/device:GPU:1')
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).


KeyError: in user code:

    File "C:\ProgramData\miniconda3\envs\tensorflow-gpu\lib\site-packages\transformers\trainer_tf.py", line 707, in distributed_training_steps  *
        nb_instances_in_batch = self._compute_nb_instances(batch)
    File "C:\ProgramData\miniconda3\envs\tensorflow-gpu\lib\site-packages\transformers\trainer_tf.py", line 714, in _compute_nb_instances  *
        labels = batch[-1]

    KeyError: -1


In [ ]:
model.save_pretrained("./bert-german-final")